# Notebook 2: Advanced Chat Capabilities with Oobabooga

## Introduction

Welcome to the advanced course! We're diving into more complex chat capabilities. Hold tight!

## Setup

Just like in Notebook 01, let's set up the environment first.

**Step 1**: Configure your Oobabooga service settings

Use [this notebook](0-AI-settings.ipynb) to save your Oobabooga settings in the configuration file.
Here, we load the settings section relevant to this notebook.

In [ ]:
// Load some helper functions, e.g. to load values from settings.json
#!import config/Settings.cs

//Import package for loading hierarchichal settings from settings.json
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0-rc.1.23419.4"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0-rc.1.23419.4"
#r "nuget: Microsoft.Extensions.Configuration.Binder, 8.0.0-rc.1.23419.4"

#!import config/OobaboogaConnectorConfiguration.cs

// Load configuration using builder package

using Microsoft.Extensions.Configuration;

var builder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("config/settings.json", optional: false, reloadOnChange: true);

IConfiguration configuration = builder.Build();

 var oobaboogaConfiguration = configuration.GetSection("Oobabooga").Get<OobaboogaConnectorConfiguration>();

**Step 2**: Import Semantic Kernel SDK and Oobabooga from NuGet

We're importing the big guns again. Semantic Kernel and Oobabooga, come on in!

In [ ]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 0.24.230918.1-preview"
// Import Oobabooga connector
#r "nuget: MyIA.SemanticKernel.Connectors.AI.Oobabooga,  0.33.1"


**Step 3**: Create Oobabooga Chat completion settings

We're setting up chat completion parameters. It's got the same parameters as text completion, including generation presets, plus all the oobabooga parameters for chat, including character presets.
We're only dealing with the basic endpoint parameters here.

In [ ]:
using MyIA.SemanticKernel.Connectors.AI.Oobabooga.Completion.ChatCompletion;

//Build settings from configuration file

var oobaboogaChatCompletionSettings = new OobaboogaChatCompletionSettings(
    endpoint: new Uri(oobaboogaConfiguration.EndPoint),
    blockingPort: oobaboogaConfiguration.BlockingPort,
    streamingPort: oobaboogaConfiguration.StreamingPort
);

// Serialize to JSON
string jsonString = JsonSerializer.Serialize(oobaboogaChatCompletionSettings);

// Display the JSON string
Console.WriteLine($"Serialized settings: {jsonString}");


## Basic Chat Completion

Let's start simple. We're adapting a basic chat test to this notebook. You'll ask a question, and Oobabooga will answer. Note that default settings use the Example character presets. Her name is Chiharu Yamada. Check here out in Oobabooga's user interface.

In [ ]:
using Microsoft.SemanticKernel.AI.ChatCompletion;
using System.Threading;

var oobaboogaLocal = new OobaboogaChatCompletion(oobaboogaChatCompletionSettings);

var history = new ChatHistory();
var message = "What is your name?";
history.AddUserMessage(message);
Console.WriteLine($"user: {message}");

var localResponse = await oobaboogaLocal.GetChatCompletionsAsync(history, new ChatRequestSettings()
{
    Temperature = 0.01,
    MaxTokens = 20,
    TopP = 0.1,
});

var chatMessage = await localResponse[^1].GetChatMessageAsync(CancellationToken.None).ConfigureAwait(false);
Console.WriteLine($"{chatMessage.Role}: {chatMessage.Content}");

## Streaming Chat Completion

Now, let's make it a bit more real-time with streaming. Note that responses bits are streamed in a funny way, so you may want to adapt that.

In [ ]:
using System.Text;
using Microsoft.SemanticKernel.AI.ChatCompletion;



ChatMessageBase? chatMessage = null;

var oobaboogaLocal = new OobaboogaChatCompletion(oobaboogaChatCompletionSettings);
var history = new ChatHistory();

var message = "What is your name?";
history.AddUserMessage(message);
Console.WriteLine($"user: {message}");

var localResponse = oobaboogaLocal.GetStreamingChatCompletionsAsync(history, new ChatRequestSettings()
{
    Temperature = 0.01,
    MaxTokens = 7,
    TopP = 0.1,
});

await foreach (var result in localResponse)
{
    await foreach (var message in result.GetStreamingChatMessageAsync())
    {
        Console.WriteLine($"{message.Role}: {message.Content}");
        chatMessage = message;
    }
}


To make the output more user-friendly, we can adapt our console writing routine to only write the new characters received. Here's how:

In [ ]:
using System.Text;
using Microsoft.SemanticKernel.AI.ChatCompletion;

StringBuilder assistantResponse = new StringBuilder();
ChatMessageBase? chatMessage = null;

var oobaboogaLocal = new OobaboogaChatCompletion(oobaboogaChatCompletionSettings);
var history = new ChatHistory();

var message = "What is your name?";
history.AddUserMessage(message);
Console.WriteLine($"user: {message}");

bool isFirstMessage = true;

var localResponse = oobaboogaLocal.GetStreamingChatCompletionsAsync(history, new ChatRequestSettings()
{
    Temperature = 0.01,
    MaxTokens = 7,
    TopP = 0.1,
});

await foreach (var result in localResponse)
{
    await foreach (var message in result.GetStreamingChatMessageAsync())
    {
        if (isFirstMessage)
        {
            Console.Write($"{message.Role}: {message.Content}");
            assistantResponse.Append(message.Content);
            isFirstMessage = false;
        }
        else
        {
            // Append only the new characters to the existing assistant's response
            var newChars = message.Content.Substring(assistantResponse.Length);
            assistantResponse.Append(newChars);
            
            // Write the new characters to the console
            Console.Write(newChars);
        }
        
        chatMessage = message;
    }
}


## Advanced Chat with Kernel

Let's see how to configure your oobabooga chat service into the kernel, and how to handle an interactive session

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.AI.ChatCompletion;
using MyIA.SemanticKernel.Connectors.AI.Oobabooga;
// using System.Diagnostics;

// Configure the two AI features: OpenAI Chat and DALL-E 2 for image generation
var builder = new KernelBuilder();

builder.WithOobaboogaChatCompletionService(oobaboogaChatCompletionSettings);

// Debugger.Break();
var kernel = builder.Build();

// Get AI service instance used to manage the user chat
var oobaboogaLocal = kernel.GetService<IChatCompletion>();

### Chat configuration

Before starting the chat, we create a new chat object with some instructions, which are included in the chat history. 

The instructions tell OpenAI what kind of chat we want to have

In [ ]:
using Microsoft.SemanticKernel.Connectors.AI.OpenAI.ChatCompletion;

var systemMessage = "You're chatting with a user. Provide helpful and accurate responses.";

var chat = oobaboogaLocal.CreateNewChat(systemMessage);


### Let's Chat

Run the following code to start the chat. The chat consists of a loop with these main steps:

1. Ask the user for a message. Add the user message into the Chat History object.
2. Send the chat object to Oobabooga asking to generate a response. Add the bot message into the Chat History object.
3. Show the answer to the user.


In [ ]:
var userMessage = await InteractiveKernel.GetInputAsync("Your message");
while (userMessage!="exit")
{
    // 1. Ask the user for a message and add it to the Chat History object.
    
    chat.AddUserMessage(userMessage);
    Console.WriteLine($"user: {userMessage}");

    // 2. Send the chat object to Oobabooga to generate a response.
        var localResponse = await oobaboogaLocal.GetChatCompletionsAsync(chat, new ChatRequestSettings(){
     Temperature = 0.01,
     MaxTokens = 20,
     TopP = 0.1,
     });
    var assistantMessage = await localResponse[^1].GetChatMessageAsync(CancellationToken.None).ConfigureAwait(false);
    chat.AddAssistantMessage(assistantMessage.Content);

    // 3. Show the answer to the user.
    Console.WriteLine($"{assistantMessage.Role}: {assistantMessage.Content}");
    userMessage = await InteractiveKernel.GetInputAsync("Your message (or type 'exit' to exit)");
}


## Conclusion and Next Steps

You've just leveled up your chatbot game! Now, you may dive into the numerous settings provided to control the connector and oobabooga, or learn playing with multiple models thanks to a routing multiconnector.